# Testing for:
## Function for Loading the Data from a REST-API for a specifiable place and time space

In [17]:
# Imports
import requests
from datetime import datetime, timedelta
import pandas as pd

# Basic parameters
base_url='http://sensordata.gwdg.de/api/' # Select Endpoint (P1 or P2)
endpoint_url_P1='measurements/P1'
endpoint_url_P2='measurements/P2'

# Select geo-coordinates (values are just examples ..)
latrange=[51,52]
longrange=[9,10]

# Select time range (values are just examples ..)
# 1st Jan 2019 - EURO 4
# 1st Jul 2020 - EUR0 5
start_date = datetime(2018, 12, 30)
end_date = (start_date + timedelta(hours=1))

# Build the query
mydata = '{"timeStart": "'+start_date.strftime("%Y-%m-%dT%H:%M:%SZ")+'",' + \
         '"timeEnd": "'+end_date.strftime("%Y-%m-%dT%H:%M:%SZ")+'", "area":  \
         {"coordinates":['+str(latrange)+','+str(longrange)+']}}'

# Run the query
print(mydata)
response_P1 = requests.post(base_url + endpoint_url_P1, data=mydata)
response_P2 = requests.post(base_url + endpoint_url_P2, data=mydata)

#response_P2.text

{"timeStart": "2018-12-30T00:00:00Z","timeEnd": "2018-12-30T01:00:00Z", "area":           {"coordinates":[[51, 52],[9, 10]]}}


In [45]:
j_P1 = response_P1.json()
del j_P1[1]                                                                                          # delete 'sensor' string, that causes errors
df_P1 = pd.DataFrame(j_P1[1], columns =j_P1[0])                                                      # put all in pandas data frame
df_P1 = df_P1.rename(columns={"P1": "measurement_PM10"})                                             # Change column name for better overview
l_P1 = list(range(len(df_P1["sensor_id"])))
for i in range(len(l_P1)):                                                                           # Adding unique measurement_id to merge P1 and P2
    l_P1[i] = str(df_P1["sensor_id"][i]) + "_" + str(df_P1["time"][i])
df_P1["measurement_id"] = l_P1
df_P1 = df_P1.reindex(columns = ["measurement_PM10", "time", "lat", "lon", "sensor_id", "measurement_id"])    # rearranging column names for better overview 

j_P2 = response_P2.json()
del j_P2[1]
df_P2 = pd.DataFrame(j_P2[1], columns =j_P2[0])
df_P2 = df_P2.rename(columns={"P2": "measurement_PM2.5"})
l_P2 = list(range(len(df_P2["sensor_id"])))
for j in range(len(l_P2)):                                                                           # Adding unique measurement_id to merge P1 and P2
    l_P2[j] = str(df_P2["sensor_id"][j]) + "_" + str(df_P2["time"][j])
df_P2["measurement_id"] = l_P2
df_P2 = df_P2.reindex(columns = ["measurement_PM2.5", "measurement_id"])

In [46]:
df_P1["measurement_id"][0]

'2903_2018-12-30T00:00:01Z'

In [47]:
df_P1

,measurement_PM10,time,lat,lon,sensor_id,measurement_id
0,NaN,2018-12-30T00:00:01Z,51.978,9.286,2903,2903_2018-12-30T00:00:01Z
1,NaN,2018-12-30T00:00:01Z,51.546,9.935,5701,5701_2018-12-30T00:00:01Z
2,7.07,2018-12-30T00:00:06Z,51.270,9.433,16662,16662_2018-12-30T00:00:06Z
3,NaN,2018-12-30T00:00:06Z,51.450,9.253,10905,10905_2018-12-30T00:00:06Z
4,NaN,2018-12-30T00:00:07Z,51.270,9.433,16663,16663_2018-12-30T00:00:07Z
...,...,...,...,...,...,...
1034,2.70,2018-12-30T00:59:33Z,51.551,9.874,13795,13795_2018-12-30T00:59:33Z
1035,NaN,2018-12-30T00:59:34Z,51.551,9.874,13796,13796_2018-12-30T00:59:34Z
1036,NaN,2018-12-30T00:59:44Z,51.523,9.910,8949,8949_2018-12-30T00:59:44Z
1037,1.93,2018-12-30T00:59:52Z,51.273,9.543,16352,16352_2018-12-30T00:59:52Z


In [48]:
df_P2

,measurement_PM2.5,measurement_id
0,NaN,2903_2018-12-30T00:00:01Z
1,NaN,5701_2018-12-30T00:00:01Z
2,2.1,16662_2018-12-30T00:00:06Z
3,NaN,10905_2018-12-30T00:00:06Z
4,NaN,16663_2018-12-30T00:00:07Z
...,...,...
1034,1.9,13795_2018-12-30T00:59:33Z
1035,NaN,13796_2018-12-30T00:59:34Z
1036,NaN,8949_2018-12-30T00:59:44Z
1037,0.9,16352_2018-12-30T00:59:52Z


In [49]:
df_total = pd.merge(df_P1, df_P2, on = "measurement_id")                                         # merge data frame on unique measurement_id
df_total = df_total.reindex(columns = ["measurement_PM10", "measurement_PM2.5", "time", "lat", "lon", "sensor_id", "measurement_id"])
df_total

,measurement_PM10,measurement_PM2.5,time,lat,lon,sensor_id,measurement_id
0,NaN,NaN,2018-12-30T00:00:01Z,51.978,9.286,2903,2903_2018-12-30T00:00:01Z
1,NaN,NaN,2018-12-30T00:00:01Z,51.546,9.935,5701,5701_2018-12-30T00:00:01Z
2,7.07,2.1,2018-12-30T00:00:06Z,51.270,9.433,16662,16662_2018-12-30T00:00:06Z
3,NaN,NaN,2018-12-30T00:00:06Z,51.450,9.253,10905,10905_2018-12-30T00:00:06Z
4,NaN,NaN,2018-12-30T00:00:07Z,51.270,9.433,16663,16663_2018-12-30T00:00:07Z
...,...,...,...,...,...,...,...
1034,2.70,1.9,2018-12-30T00:59:33Z,51.551,9.874,13795,13795_2018-12-30T00:59:33Z
1035,NaN,NaN,2018-12-30T00:59:34Z,51.551,9.874,13796,13796_2018-12-30T00:59:34Z
1036,NaN,NaN,2018-12-30T00:59:44Z,51.523,9.910,8949,8949_2018-12-30T00:59:44Z
1037,1.93,0.9,2018-12-30T00:59:52Z,51.273,9.543,16352,16352_2018-12-30T00:59:52Z


## Giving out duplicated values with their frequency

In [6]:
df_P1.pivot_table(index=['sensor_id'], aggfunc='size')

sensor_id
10904     1
10905    13
1112     24
1113     24
11998    24
11999    24
12000    24
12304     2
12305    20
12334    20
12335    20
12972    22
12973    24
13795    25
13796    25
16177    23
16178    23
16352    25
16353    25
16362    23
16363    23
16364    23
16662    18
16663    18
16664    18
18150    24
18935    12
18936    12
2574     24
2902     24
2903     25
3819     24
4412     24
4413     24
463       6
4957     24
4974     21
5257     56
5258     56
5608     24
5609     24
5700     24
5701     25
7828     11
8949     16
9870     24
9946     24
dtype: int64

## Giving out pivot table to generate time series

In [51]:
df_pivot = df_total.pivot(index="sensor_id", columns="time", values = "measurement_PM10")
df_pivot

time,2018-12-30T00:00:01Z,2018-12-30T00:00:06Z,2018-12-30T00:00:07Z,2018-12-30T00:00:08Z,2018-12-30T00:00:20Z,2018-12-30T00:00:21Z,2018-12-30T00:00:25Z,2018-12-30T00:00:28Z,2018-12-30T00:00:31Z,2018-12-30T00:00:32Z,...,2018-12-30T00:59:16Z,2018-12-30T00:59:17Z,2018-12-30T00:59:22Z,2018-12-30T00:59:23Z,2018-12-30T00:59:26Z,2018-12-30T00:59:33Z,2018-12-30T00:59:34Z,2018-12-30T00:59:44Z,2018-12-30T00:59:52Z,2018-12-30T00:59:53Z
sensor_id,,,,,,,,,,,,,,,,,,,,,
10904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
# Giving mean of PM10 measurements over time per sensor
df_pivot.mean(axis = 1)

sensor_id
10904    3.270000
10905         NaN
1112     4.857917
1113          NaN
11998    2.719583
11999         NaN
12000         NaN
12304    2.185000
12305         NaN
12334    3.526500
12335         NaN
12972    3.405000
12973         NaN
13795    3.763600
13796         NaN
16177    5.323043
16178         NaN
16352    2.461200
16353         NaN
16362    1.161739
16363         NaN
16364         NaN
16662    3.894444
16663         NaN
16664         NaN
18150    3.866250
18935    3.753333
18936         NaN
2574     3.835833
2902     3.483750
2903          NaN
3819     4.100417
4412     3.792083
4413          NaN
463           NaN
4957     3.404167
4974          NaN
5257     1.750714
5258          NaN
5608     6.233750
5609          NaN
5700     3.452917
5701          NaN
7828          NaN
8949          NaN
9870     3.667083
9946     5.544583
dtype: float64